In [1]:
# this script should be run after postprocess fixation video 
# the output is a file called dorsal_videos_dataset.pkl, containing the complete dataset that can be used for model fitting
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from os.path import join
import os

dates = ["082824", "082924", "083024", "012925", "013025", "013125", "020425", "021125"]

monitor_latency = 0.03
start_win = 0.04

reliability_cutoff = 0.5
firing_rate_cutoff = 2

start_time = start_win + monitor_latency

load_dir = f"./neurips/preprocessed/"

times = np.load(join(load_dir, f"times_video.npy"))

with open(join(load_dir, f"neuron_data.pickle"), 'rb') as f:
    neuron_df = pickle.load(f)

session_dfs = {}
sessions = []
neuron_ids = []

spikes = {}

for date in dates:
    session_dfs[int(date)] = pd.read_csv(join(load_dir, f"{date}_video.csv"))
    subset_df = neuron_df.loc[neuron_df['session_id'] == int(date)]
    for neuron_id in range(subset_df['neuron_id'].max()+1):
        sessions.append(int(date))
        neuron_ids.append(neuron_id)
        
    spikes[int(date)] = np.load(join(load_dir, f"{date}_video.npy"))# np.transpose(np.array(spike_trains), (1, 2, 0))

session_ids = np.array(sessions)
neuron_ids = np.array(neuron_ids)

In [2]:
import pickle
import os 

with open(join(load_dir, f"neuron_data.pickle"), 'rb') as handle:
    neuron_data_df = pickle.load(handle)

In [3]:
neuron_data_df.head()

,session_id,neuron_id,x_coord_neu,y_coord_neu,template_times,template,template_trough_to_peak,template_type,rf_x_cond_label,rf_y_cond_label,rf,rf_norm,x_coord_rf,y_coord_rf,monkey_id,px_per_deg,fix_coord_deg
0,82824,0,11.0,20.0,"[0.0, 0.03333333333333333, 0.06666666666666667...","[0.22548859, 0.19542037, 0.18996716, 0.1756644...",366.666667,RS,"[-3, 0, 3, 6, 9, 12, 15]","[-15, -12, -9, -6, -3, 0, 3, 6, 9, 12, 15]","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",NaN,NaN,T,10.075732,[-5 0]
1,82824,1,11.0,20.0,"[0.0, 0.03333333333333333, 0.06666666666666667...","[-0.03260965, -0.052709486, -0.06782218, -0.07...",200.0,FS,"[-3, 0, 3, 6, 9, 12, 15]","[-15, -12, -9, -6, -3, 0, 3, 6, 9, 12, 15]","[[0.0015384615384615385, 0.002857142857142857,...","[[0.2884615384615385, 0.5357142857142858, 0.40...",12.0,-9.0,T,10.075732,[-5 0]
2,82824,2,11.0,0.0,"[0.0, 0.03333333333333333, 0.06666666666666667...","[-0.27105498, -0.2881631, -0.3233422, -0.30338...",-466.666667,AS,"[-3, 0, 3, 6, 9, 12, 15]","[-15, -12, -9, -6, -3, 0, 3, 6, 9, 12, 15]","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0007692...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5769230...",6.0,-9.0,T,10.075732,[-5 0]
3,82824,3,114.0,20.0,"[0.0, 0.03333333333333333, 0.06666666666666667...","[-0.14516835, -0.16483098, -0.19752464, -0.184...",-366.666667,AS,"[-3, 0, 3, 6, 9, 12, 15]","[-15, -12, -9, -6, -3, 0, 3, 6, 9, 12, 15]","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",6.0,-3.0,T,10.075732,[-5 0]
4,82824,4,11.0,0.0,"[0.0, 0.03333333333333333, 0.06666666666666667...","[0.25014895, 0.25289863, 0.26891083, 0.2699676...",433.333333,RS,"[-3, 0, 3, 6, 9, 12, 15]","[-15, -12, -9, -6, -3, 0, 3, 6, 9, 12, 15]","[[0.0023076923076923075, 0.0014285714285714286...","[[0.2937062937062937, 0.18181818181818185, 0.1...",9.0,-3.0,T,10.075732,[-5 0]


In [4]:
def get_model_rates(session_id, time_window=0.2, start_time = 0.1):    
    test_df = session_dfs[session_id]
    spike_train = spikes[session_id]
    
    n_images = spike_train.shape[0]
    n_neurons = spike_train.shape[2]
    
    fr = np.empty((n_images, n_neurons))
    fr[:] = np.nan
    
    end_time = start_time + time_window
    time_idx = (times > start_time) & (times < end_time)
    
    assert n_images == test_df.shape[0]
    
    for i in range(n_images):
        fr[i, :] = np.mean(spike_train[i, time_idx, :], axis=0)*1000 # spikes/sec

    return fr

In [5]:
model_inputs = dict()
for session_id in dates:
    rates = get_model_rates(int(session_id), start_time = start_time)
    images = session_dfs[int(session_id)]['image'].to_numpy()
    train = session_dfs[int(session_id)]['train'].to_numpy()
    
    model_input = dict()
    model_input["images"] = images
    model_input["rates"] = rates
    model_input["split"] = train.astype(bool)
    model_inputs[session_id] = model_input

In [6]:
all_rates = []
all_splits = []
all_images = []
base_rates = []
base_test_rates = []

for session_id in dates:
    model_input = model_inputs[session_id]
    
    rates = model_input["rates"]
    images = model_input["images"]
    split = model_input["split"]
    
    # zscore rates based on the train set mean and std
    train_rates = rates[split]
    mean_rate = np.nanmean(train_rates, 0, keepdims=True)
    base_rates.append(np.nanmean(train_rates, 0, keepdims=True))
    base_test_rates.append(np.nanmean(rates[np.logical_not(split)], 0, keepdims=True))
    std_rate = np.nanstd(train_rates, 0, keepdims=True)
    zscored_rates = (rates - mean_rate)/std_rate
    
    all_rates.append(zscored_rates)
    all_splits.append(split)
    all_images.append(images)
    
mean_train_rates = np.column_stack(base_rates).T[:, 0]
mean_test_rates = np.column_stack(base_test_rates).T[:, 0]

/tmp/ipykernel_192491/672678579.py:20: RuntimeWarning: divide by zero encountered in divide
  zscored_rates = (rates - mean_rate)/std_rate
/tmp/ipykernel_192491/672678579.py:20: RuntimeWarning: invalid value encountered in divide
  zscored_rates = (rates - mean_rate)/std_rate


In [7]:
# get list of train and test stimuli by aggregating stimuli shown over sessions
train_stims = []
test_stims = []

for i in range(len(all_images)):
    split = all_splits[i]
    curr_train_images = all_images[i][split]
    curr_test_images = all_images[i][np.logical_not(split)]
    
    for image in curr_train_images:
        if image not in train_stims: 
            train_stims = train_stims + [image]
    
    for image in curr_test_images:
        if image not in test_stims:
            test_stims = test_stims + [image]

In [8]:
# count total number of neurons
n_neurons = 0
for i in range(len(all_rates)):
    n_neurons += all_rates[i].shape[1]

In [9]:
assert n_neurons == neuron_data_df.shape[0]

In [10]:
n_train = len(train_stims)
n_test = len(test_stims)
train_activity = np.full((n_train, n_neurons), np.nan)
test_activity = np.full((n_test, n_neurons), np.nan)

In [11]:
from tqdm import trange
import numpy as np
test_stims = np.array(test_stims)
train_stims = np.array(train_stims)

neu_cnt = 0
for i in trange(len(all_rates)):
    curr_rates = all_rates[i]
    curr_stims = np.array(all_images[i])
    
    train_dict = {}
    test_dict = {}
    num_neus = curr_rates.shape[1]
    for k in range(curr_rates.shape[0]):
        stim = curr_stims[k]
        if stim in train_stims:
            train_activity[train_stims == stim, neu_cnt:num_neus+neu_cnt] = curr_rates[k, :]
        elif stim in test_stims:
            assert(len(np.mean(curr_rates[curr_stims == stim, :], 0)) == num_neus)
            test_activity[test_stims == stim, neu_cnt:num_neus+neu_cnt] = np.mean(curr_rates[curr_stims == stim, :], 0)            
        else:
            raise ValueError()
    neu_cnt = num_neus + neu_cnt


100%|██████████| 8/8 [00:01<00:00,  5.01it/s]


In [12]:
#session_ids, neuron_ids maintain correct mapping between train_activity, test_activity and contents of the df

In [13]:
from tqdm import trange
import numpy as np

max_reps = 50
test_activity_reps = np.full((n_test, n_neurons, max_reps), np.nan)

neu_cnt = 0
for i in trange(len(all_rates)):
    rep_cnts = {}
    for stim in test_stims:
        rep_cnts[stim] = 0
        
    curr_rates = all_rates[i]
    curr_stims = np.array(all_images[i])
    
    train_dict = {}
    test_dict = {}
    num_neus = curr_rates.shape[1]
    for k in range(curr_rates.shape[0]):
        stim = curr_stims[k]
        if stim in test_stims:
            test_activity_reps[test_stims == stim, neu_cnt:num_neus+neu_cnt, rep_cnts[stim]] = curr_rates[k, :]
            rep_cnts[stim] += 1
            
    neu_cnt = num_neus + neu_cnt

100%|██████████| 8/8 [00:00<00:00, 83.52it/s]


In [14]:
# response reliability - 50 bootstraps
seed = 5
np.random.seed(seed)
n_bootstraps = 50
reliability = np.full((n_neurons), np.nan)

for neu_idx in range(test_activity_reps.shape[1]):
    row_subset = np.sum(np.isnan(test_activity_reps[:, neu_idx, :]), 0) != n_test
    curr_act = test_activity_reps[:, neu_idx, row_subset]
    rs = np.full((n_bootstraps), np.nan)
    for bootstrap_idx in range(n_bootstraps):
        length = curr_act.shape[1]
        indices = np.arange(length)
        np.random.shuffle(indices)
        midpoint = length // 2
        act1 = np.nanmean(curr_act[:, indices[:midpoint]], 1)
        act2 = np.nanmean(curr_act[:, indices[midpoint:]], 1)
        nan_idx = np.isnan(act1) | np.isnan(act2)
        act1 = act1[~nan_idx]
        act2 = act2[~nan_idx]
        
        rs[bootstrap_idx] = np.corrcoef(act1, act2)[0, 1]

    reliability[neu_idx] = np.nanmean(rs)

/home/groups/guosongh/usr/local/lib/python3.9/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/groups/guosongh/usr/local/lib/python3.9/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/tmp/ipykernel_192491/465570026.py:16: RuntimeWarning: Mean of empty slice
  act1 = np.nanmean(curr_act[:, indices[:midpoint]], 1)
/tmp/ipykernel_192491/465570026.py:17: RuntimeWarning: Mean of empty slice
  act2 = np.nanmean(curr_act[:, indices[midpoint:]], 1)
/home/groups/guosongh/usr/local/lib/python3.9/site-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/home/groups/guosongh/usr/local/lib/python3.9/site-packages/numpy/core/_methods.py:121: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/home/groups/guosongh/usr/local/lib/python3.9/site-packages/numpy/lib/fun

In [15]:
# criteria for determining which neurons to include
include_idx = (reliability > reliability_cutoff) & (mean_train_rates > firing_rate_cutoff) & (mean_test_rates != 0)
print(f'total num of neurons: {np.sum(include_idx)}')

total num of neurons: 2244


In [16]:
#session_ids, neuron_ids maintain correct mapping between train_activity, test_activity and contents of the df
# so to get a uid for the neuron, we need to do the following
session_ids = session_ids[include_idx]
neuron_ids = neuron_ids[include_idx]
neuron_uids = np.array(np.arange(np.sum(include_idx)), dtype=int)

In [17]:
# save neuron_df
df = pd.DataFrame({
    'neuron_id': neuron_ids,
    'session_id': session_ids,
    'neuron_uid': neuron_uids,
})

In [18]:
df['session_id'] = df['session_id'].astype(int)
df = df.merge(neuron_data_df, left_on=["session_id", "neuron_id"], right_on=["session_id", "neuron_id"])

In [19]:
data = dict()
data['reliability'] = reliability[include_idx]
data['train_activity'] = train_activity[:, include_idx]
data['test_activity'] = test_activity[:, include_idx]
data['train_stimuli'] =  train_stims
data['test_stimuli'] =  test_stims

In [20]:
import pickle

dorsal_path = f"./dataset/dorsal_stream_dataset.pickle"
dorsal_neu_path = f"./dataset/dorsal_stream_neuron_table.pickle"

def save_data(data, path):
    with open(path, 'wb') as f:
        pickle.dump(data, f)

save_data(data, dorsal_path)
save_data(df, dorsal_neu_path)